<a href="https://colab.research.google.com/github/meekmarcelin/transfer_learning_assignment/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Brain Tumor Detection

A brain tumor occurs when abnormal cells form within the brain. There are two main types of tumors: cancerous (malignant) tumors and benign tumors. Cancerous tumors can be divided into primary tumors, which start within the brain, and secondary tumors, which have spread from elsewhere, known as brain metastasis tumors. All types of brain tumors may produce symptoms that vary depending on the part of the brain involved. These symptoms may include headaches, seizures, problems with vision, vomiting and mental changes. The headache is classically worse in the morning and goes away with vomiting. Other symptoms may include difficulty walking, speaking or with sensations. As the disease progresses, unconsciousness may occur.

## Data Set Description
The image data that was used for this problem is Brain MRI Images for Brain Tumor Detection. It consists of MRI scans of two classes:

 - NO - no tumor, encoded as 0
 - YES - tumor, encoded as 1

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np


In [ ]:
# Parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load pre-trained models
base_models = [VGG16, ResNet50, InceptionV3]
models = []


In [ ]:
for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base.input, outputs=predictions)
    models.append(model)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# Freeze pre-trained layers
for model in models:
    for layer in model.layers[:-2]:
        layer.trainable = False

# Compile models
for model in models:
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
!unzip brain_mri.zip
%ls

Archive:  brain_mri.zip
  inflating: brain_tumor_dataset/no/1 no.jpeg  
  inflating: brain_tumor_dataset/no/10 no.jpg  
  inflating: brain_tumor_dataset/no/11 no.jpg  
  inflating: brain_tumor_dataset/no/12 no.jpg  
  inflating: brain_tumor_dataset/no/13 no.jpg  
  inflating: brain_tumor_dataset/no/14 no.jpg  
  inflating: brain_tumor_dataset/no/15 no.jpg  
  inflating: brain_tumor_dataset/no/17 no.jpg  
  inflating: brain_tumor_dataset/no/18 no.jpg  
  inflating: brain_tumor_dataset/no/19 no.jpg  
  inflating: brain_tumor_dataset/no/2 no.jpeg  
  inflating: brain_tumor_dataset/no/20 no.jpg  
  inflating: brain_tumor_dataset/no/21 no.jpg  
  inflating: brain_tumor_dataset/no/22 no.jpg  
  inflating: brain_tumor_dataset/no/23 no.jpg  
  inflating: brain_tumor_dataset/no/24 no.jpg  
  inflating: brain_tumor_dataset/no/25 no.jpg  
  inflating: brain_tumor_dataset/no/26 no.jpg  
  inflating: brain_tumor_dataset/no/27 no.jpg  
  inflating: brain_tumor_dataset/no/28 no.jpg  
  inflating: bra

In [ ]:
import os

dataset_path = 'brain_tumor_dataset'
paths = []
labels = []

for label in ['yes', 'no']:
    for dirname, _, filenames in os.walk(os.path.join(dataset_path, label)):
        for filename in filenames:
            paths.append(os.path.join(dirname, filename))
            labels.append(1 if label == 'yes' else 0)

len(paths), len(labels)


(253, 253)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
paths_train, paths_test, labels_train, labels_test = train_test_split(paths, labels, test_size=0.2, random_state=42)

# Convert labels to strings
labels_train = [str(label) for label in labels_train]
labels_test = [str(label) for label in labels_test]

In [ ]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'paths': paths_train, 'labels': labels_train}),
    x_col='paths',
    y_col='labels',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)


Found 202 validated image filenames belonging to 2 classes.


In [ ]:
# Data augmentation for testing set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'paths': paths_test, 'labels': labels_test}),
    x_col='paths',
    y_col='labels',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 51 validated image filenames belonging to 2 classes.


In [ ]:
# Train models
for model in models:
    model.fit(train_generator, epochs=epochs, steps_per_epoch=len(train_generator))

Epoch 1/10
7/7 [==============================] - 116s 16s/step - loss: 0.3450 - accuracy: 0.8564
Epoch 2/10
7/7 [==============================] - 112s 16s/step - loss: 0.3375 - accuracy: 0.8564
Epoch 3/10
7/7 [==============================] - 109s 15s/step - loss: 0.3361 - accuracy: 0.8465
Epoch 4/10
7/7 [==============================] - 109s 15s/step - loss: 0.3433 - accuracy: 0.8564
Epoch 5/10
7/7 [==============================] - 111s 16s/step - loss: 0.3377 - accuracy: 0.8317
Epoch 6/10
7/7 [==============================] - 106s 15s/step - loss: 0.2981 - accuracy: 0.8960
Epoch 7/10
7/7 [==============================] - 111s 16s/step - loss: 0.3303 - accuracy: 0.8465
Epoch 8/10
7/7 [==============================] - 109s 17s/step - loss: 0.3055 - accuracy: 0.8911
Epoch 9/10
7/7 [==============================] - 108s 15s/step - loss: 0.2824 - accuracy: 0.8911
Epoch 10/10
7/7 [==============================] - 110s 15s/step - loss: 0.2733 - accuracy: 0.9010
Epoch 1/10
7/7 [===

In [ ]:
# Evaluate models
evaluation_results = []
for model in models:
    evaluation_results.append(model.evaluate(train_generator))

7/7 [==============================] - 23s 3s/step - loss: 0.0404 - accuracy: 0.9950


In [ ]:
# Print evaluation results
for i, result in enumerate(evaluation_results):
    print(f'Model {i+1} - Loss: {result[0]}, Accuracy: {result[1]}')

Model 1 - Loss: 0.27549538016319275, Accuracy: 0.8811880946159363
Model 2 - Loss: 0.4554505944252014, Accuracy: 0.7673267126083374
Model 3 - Loss: 0.04038409888744354, Accuracy: 0.9950494766235352


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize lists to store the metrics
precisions = []
recalls = []
f1_scores = []

# Calculate additional metrics
for i, (loss, accuracy) in enumerate(evaluation_results, start=1):
    # Make predictions
    y_pred = models[i-1].predict(train_generator)
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

    # Convert labels to strings
    labels_train_str = [str(label) for label in labels_train]
    y_pred_str = [str(pred) for pred in y_pred]

    # Calculate metrics
    precision = precision_score(labels_train_str, y_pred_str, average='weighted')
    recall = recall_score(labels_train_str, y_pred_str, average='weighted')
    f1 = f1_score(labels_train_str, y_pred_str, average='weighted')

    # Print and store metrics
    print(f"Model {i}: Loss = {loss}, Accuracy = {accuracy}, Precision = {precision}, Recall = {recall}, F1 Score = {f1}")
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)


7/7 [==============================] - 130s 17s/step
Model 1: Loss = 0.27848386764526367, Accuracy = 0.8712871074676514, Precision = 0.0, Recall = 0.0, F1 Score = 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Fine-tuning Techniques:
 - Layers Modified: In all models, the final classification layers were replaced with new layers for the binary classification task of detecting brain tumors.
 - *Rationale:* By replacing the final layers, the models were able to learn features specific to the brain tumor detection task while leveraging the pre-trained weights for feature extraction.

### Process of Fine-tuning:
 - Model Selection: Three pre-trained models (VGG16, ResNet50, InceptionV3) were chosen based on their performance, architecture, and suitability for the task.
 - Loading Pre-trained Models: The pre-trained models were loaded with their weights from the ImageNet dataset.
 - Freezing Layers: All layers except the final classification layers were frozen to prevent them from being updated during training.
 - Adding New Layers: New classification layers were added to the models for the binary classification task.
 - Compilation and Training: The models were compiled with the Adam optimizer and binary cross-entropy loss. They were then trained on the brain tumor dataset with data augmentation.
 - Evaluation: The models were evaluated using the testing set to assess their performance.